# Inputs de Dados

In [ ]:
##Importando Bibliotecas

import pandas as pd
import re

In [ ]:
##Importando base de dados

base = pd.read_csv('Indicadores Covid (Teste Técnico DQ).csv', sep=',', encoding='utf-8')

In [ ]:
##Criando catalogo de UFs validas conforme padrão IBGE 

ufs_validas = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']

# Conhecendo a base de dados

In [ ]:
## Visualizando primeiras linhas
base.head(3)

In [ ]:
## Visualizando uma amostra aleatória
base.sample(3)

In [ ]:
## Visualizando ultimas linhas
base.tail(3)

In [ ]:
## Tamanho da base
base.shape

In [ ]:
## Nome das colunas
base.columns

In [ ]:
## Informações gerais
base.info()

In [ ]:
## Valores nulos
base.isnull().sum()

# Criando funções de verificação de inconsistencias

In [ ]:
## Verifica preenchimento

def verificar_preenchimento(row):
    return 'consistente' if row.notnull().all() and (row != '').all() else 'inconsistente'

In [ ]:
## Verifica Padrão

def verificar_padronizacao(row):
    padronizacao = {
       'sintomas': '^[A-Z]+$',
       'profissionalSaude': '^(SIM|NAO)$',
       'racaCor': '^(AMARELA|BRANCA|IGNORADO|INDIGENA|PARDA|PRETA)$',
       'outrosSintomas': '^[A-Z]+$',
       'sexo': '^(MASCULINO|FEMININO|INDEFINIDO)$',
       'uf': '^[A-Z]{2}$',
       'municipio': '^[A-Z]+$',
       'codigoLaboratorioPrimeiraDose': '^(ASTRAZENECA/FIOCRUZ|JANSSEN|SINOVAC/BUTANTAN|PFIZER)$',
       'codigoLaboratorioSegundaDose': '^(ASTRAZENECA/FIOCRUZ|JANSSEN|SINOVAC/BUTANTAN|PFIZER)$',
       'dataNotificacao': '^\d{4}-\d{2}-\d{2}$',
       'dataInicioSintomas': '^\d{4}-\d{2}-\d{2}$',
       'dataPrimeiraDose': '^\d{4}-\d{2}-\d{2}$',
       'dataSegundaDose': '^\d{4}-\d{2}-\d{2}$',
       'Idade': '^\d+$'
    }
    for col, regex in padronizacao.items():
        if pd.notnull(row[col]) and not re.fullmatch(regex, row[col]):
            return 'inconsistente'
    return 'consistente'

In [ ]:
## Verifica Consistencia

def verificar_consistencia(row):
    if pd.notnull(row['dataNotificacao']) and not (pd.to_datetime('2020-01-04') <= pd.to_datetime(row['dataNotificacao']) <= pd.to_datetime('2022-07-22')):
        return 'inconsistente'
    if pd.notnull(row['dataInicioSintomas']) and not (pd.to_datetime('2020-01-04') <= pd.to_datetime(row['dataInicioSintomas']) <= pd.to_datetime('2022-07-22')):
        return 'inconsistente'
    if pd.notnull(row['dataPrimeiraDose']) and not (pd.to_datetime('2020-03-23') <= pd.to_datetime(row['dataPrimeiraDose']) <= pd.to_datetime('2022-07-22')):
        return 'inconsistente'
    if pd.notnull(row['dataSegundaDose']) and not (pd.to_datetime('2020-08-19') <= pd.to_datetime(row['dataSegundaDose']) <= pd.to_datetime('2022-07-22')):
        return 'inconsistente'
    if (pd.notnull(row['dataPrimeiraDose']) or pd.notnull(row['dataSegundaDose'])) and pd.to_datetime(row['dataPrimeiraDose']) > pd.to_datetime(row['dataSegundaDose']):
        return 'inconsistente'
    if pd.notnull(row['idade']) and (row['idade'] < 0 or row['idade'] >= 122):
        return 'inconsistente'
    return 'consistente'

In [ ]:
## Verifica duplicidade

def verifica_duplicidade(df, valor):
    return 'inconsistente' if df['source_id'].duplicated().sum() == 0 else 'consistente'

In [ ]:
## Verifica abrangencia

def verifica_abrangencia(valor, ufs_validas):
    return 'consistente' if valor in ufs_validas else 'inconsistente'

# Aplicando funções de verificação de inconsistencias

In [ ]:
base['preenchimento'] = base.apply(lambda row: verificar_preenchimento(row), axis=1)
base['padronizacao'] = base.apply(lambda row: verificar_padronizacao(row), axis=1)
base['consistencia'] = base.apply(lambda row: verificar_consistencia(row), axis=1)
base['unicidade'] = verifica_duplicidade(base, base['source_id'])
base['abrangencia'] = base.apply(lambda row: verifica_abrangencia(row['uf'], ufs_validas), axis=1)

# Verificando a base de dados modelada

In [ ]:
## Tamanho da base
base.shape

In [ ]:
## Nome das colunas
base.columns

In [ ]:
## Tipo de dados
base.dtypes

# Salvando base modelada

In [ ]:
base.to_csv('baseModelada.csv', sep=';',encoding='latin', index=False)